### SEP a analizar 
<div>
<img src="SEP.png" width="500"/>
</div>


### Creación del SEP en pandapower


In [61]:
import pandapower as pp
import numpy as np
from scipy.sparse import csr_matrix
import pandas as pd

#Crear red vacía
net = pp.create_empty_network()

#Crear barras
bus1 = pp.create_bus(net, vn_kv=220, name="Barra 1")
bus2 = pp.create_bus(net, vn_kv=110, name="Barra 2")
bus1A = pp.create_bus(net, vn_kv=110, name="Barra 1A")
bus2A = pp.create_bus(net, vn_kv=110, name="Barra 2A")
bus3A = pp.create_bus(net, vn_kv=110, name="Barra 3A")
bus1B = pp.create_bus(net, vn_kv=110, name="Barra 1B")
bus2B = pp.create_bus(net, vn_kv=110, name="Barra 2B")

#Crear barra slack en Barra 1
pp.create_ext_grid(net, bus1, vm_pu=1.0, va_degree=0.0, name="Slack")

#Crear transformador entre Barra 1 y Barra 2
pp.create_transformer(net, hv_bus=bus2, lv_bus=bus1, std_type="100 MVA 220/110 kV", name="Trafo 1-2")

#Crear líneas de transmisión
pp.create_line(net, from_bus=bus2, to_bus=bus1A, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 2-1A")
pp.create_line(net, from_bus=bus1A, to_bus=bus2A, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 1A-2A")
pp.create_line(net, from_bus=bus2A, to_bus=bus3A, length_km=20, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 2A-3A")
pp.create_line(net, from_bus=bus1B, to_bus=bus2, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 2-1B")
pp.create_line(net, from_bus=bus2B, to_bus=bus1B, length_km=30, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 1B-2B")
pp.create_line(net, from_bus=bus3A, to_bus=bus2B, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Line 3A-2B")

#Crear cargas
pp.create_load(net, bus2, p_mw=150, q_mvar=100, name="Load 2")  # Carga nominal en Barra 2
pp.create_load(net, bus1A, p_mw=30, q_mvar=20, name="Load 1A")
pp.create_load(net, bus2A, p_mw=52.5, q_mvar=35, name="Load 2A")
pp.create_load(net, bus3A, p_mw=22.5, q_mvar=15, name="Load 3A")
pp.create_load(net, bus2B, p_mw=90, q_mvar=60, name="Load 2B")
pp.create_load(net, bus1B, p_mw=15, q_mvar=10, name="Load 1B")



# Ejecutar el flujo de carga
pp.runpp(net)

# Acceder a la matriz Ybus directamente desde el resultado del flujo de carga
Ybus = net._ppc['internal']['Ybus']

# Configurar NumPy para mostrar solo 3 decimales
np.set_printoptions(precision=3, suppress=True)

# Convertir Ybus a un array de NumPy
Ybus_array = Ybus.toarray()

# Crear un DataFrame de pandas con la matriz Ybus
Ybus_df = pd.DataFrame(Ybus_array)

# Configurar pandas para mejorar la visualización
pd.set_option('display.max_columns', None)  # Asegura mostrar todas las columnas
pd.set_option('display.width', 1000)  # Ajusta el ancho para la visualización
pd.set_option('display.float_format', '{:.3f}'.format)  # Formato de los números flotantes

# Mostrar el DataFrame
print("Matriz de Admitancia Ybus:")
print(Ybus_df)

# Paso 1: Obtener las dimensiones de Ybus_array
num_rows, num_cols = Ybus_array.shape

# Paso 2: Crear matrices vacías para las partes real e imaginaria
Ybus_real = np.zeros((num_rows, num_cols))
Ybus_imag = np.zeros((num_rows, num_cols))

# Paso 3: Iterar sobre cada elemento de Ybus_array y separar las partes real e imaginaria
for i in range(num_rows):
    for j in range(num_cols):
        Ybus_real[i, j] = Ybus_array[i, j].real
        Ybus_imag[i, j] = Ybus_array[i, j].imag


#Voltajes iniciales (pu) y ángulos iniciales (radianes)
V = np.array([220*10**3, 0, 0, 0, 0, 0, 0])  # Incluyendo la barra slack
theta = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])  # Incluyendo la barra slack

#Matriz de admitancia Y_bus (parte real e imaginaria)
Y_real = Ybus_real

Y_imag = Ybus_imag

# Ajuste de los arrays P_inyectado y Q_inyectado para incluir todas las barras PQ
P_inyectado = np.array([0, 150*10**6, 30*10**6, 52.5*10**6, 22.5*10**6, 15*10**6, 90*10**6])  
Q_inyectado = np.array([0, 100*10**6, 20*10**6, 35*10**6, 15*10**6, 10*10**6, 60*10**6])  

#Función para calcular las derivadas parciales H, N, M, L según las definiciones dadas
def calcular_derivadas(V, theta, Y_real, Y_imag):
    n = len(V)
    H = np.zeros((n, n))
    N = np.zeros((n, n))
    M = np.zeros((n, n))
    L = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i != j:
                H[i, j] = -V[i] * V[j] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))
                N[i, j] = -V[i] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
                M[i, j] = V[i] * V[j] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
                L[i, j] = -V[i] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))
            else:
                H[i, i] = Y_imag[i, i] * V[i]**2
                N[i, i] = -Y_real[i, i] * V[i]
                M[i, i] = -Y_imag[i, i] * V[i]
                L[i, i] = -Y_real[i, i] * V[i]**2

    return H, N, M, L

#Función para calcular el vector f(x) (delta P y delta Q)
def calcular_f(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado):
    n = len(V)
    delta_P = np.zeros(n)
    delta_Q = np.zeros(n)

    for i in range(n):
        sum_P = 0.0
        sum_Q = 0.0
        for j in range(n):
            sum_P += V[i] * V[j] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
            sum_Q += V[i] * V[j] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))

        delta_P[i] = P_inyectado[i] - sum_P
        delta_Q[i] = Q_inyectado[i] - sum_Q

    f_x = np.concatenate((delta_P, delta_Q))
    return f_x
    #Función para calcular el Jacobiano J según las definiciones dadas
def calcular_jacobiano(V, theta, Y_real, Y_imag):
    n = len(V)
    H, N, M, L = calcular_derivadas(V, theta, Y_real, Y_imag)

    J = np.zeros((2 * n, 2 * n))

    # Matrices H, N, M, L en el Jacobiano J
    J[:n, :n] = H
    J[:n, n:] = N
    J[n:, :n] = M
    J[n:, n:] = L

    return J

#Función para ejecutar el método de Newton-Raphson
def newton_raphson(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado, max_iter=100, tol=1e-6):
    n = len(V)
    iteracion = 0
    delta = np.inf

    while delta > tol and iteracion < max_iter:
        # Calcular el Jacobiano J
        J = calcular_jacobiano(V, theta, Y_real, Y_imag)

        # Calcular el vector f(x) (delta P y delta Q)
        f_x = calcular_f(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado)
        J_pseudo_inv = np.linalg.pinv(J)
        delta_x = -np.dot(J_pseudo_inv, f_x)
        # Calcular delta_x usando la inversa de J
        #delta_x = -np.linalg.solve(J, f_x)

        # Actualizar V y theta
        V += delta_x[:n]
        theta += delta_x[n:]

        # Calcular la norma del vector f(x) para criterio de convergencia
        delta = np.linalg.norm(f_x)

        # Incrementar contador de iteraciones
        iteracion += 1

    if delta <= tol:
        print(f"Convergencia alcanzada en {iteracion} iteraciones.")
    else:
        print("El método de Newton-Raphson no convergió después de", max_iter, "iteraciones.")

    return V, theta

# Llamar a la función newton_raphson
V_final, theta_final = newton_raphson(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado)

# Imprimir los resultados
print("Voltajes finales (pu):", V_final)
print("Ángulos finales (radianes):", theta_final)

Matriz de Admitancia Ybus:
                 0                   1                   2                   3                   4                   5                   6
0 72.277-3332.575j    -18.042+833.132j        0.000+0.000j        0.000+0.000j        0.000+0.000j        0.000+0.000j        0.000+0.000j
1 -18.042+833.132j 7022.709-11262.503j -3509.096+5529.484j        0.000+0.000j        0.000+0.000j -3509.096+5529.484j        0.000+0.000j
2     0.000+0.000j -3509.096+5529.484j  5848.493-9209.867j -2339.397+3686.323j        0.000+0.000j        0.000+0.000j        0.000+0.000j
3     0.000+0.000j        0.000+0.000j -2339.397+3686.323j  4093.945-6442.749j -1754.548+2764.742j        0.000+0.000j        0.000+0.000j
4     0.000+0.000j        0.000+0.000j        0.000+0.000j -1754.548+2764.742j  4093.945-6442.749j        0.000+0.000j -2339.397+3686.323j
5     0.000+0.000j -3509.096+5529.484j        0.000+0.000j        0.000+0.000j        0.000+0.000j  4678.794-7363.142j -1169.699+1843.161j


C:\Users\pedro\AppData\Local\Temp\ipykernel_3240\2726376863.py:126: RuntimeWarning: overflow encountered in scalar multiply
  sum_P += V[i] * V[j] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
C:\Users\pedro\AppData\Local\Temp\ipykernel_3240\2726376863.py:127: RuntimeWarning: overflow encountered in scalar multiply
  sum_Q += V[i] * V[j] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))


UFuncTypeError: Cannot cast ufunc 'add' output from dtype('float64') to dtype('int32') with casting rule 'same_kind'

In [ ]:

# Crear un DataFrame para almacenar los resultados de las barras
barra_resultados = pd.DataFrame(columns=["Tipo de Barra", "P (MW)", "Q (MVAr)", "V (p.u.)", "Ángulo (°)"])

# Obtener los resultados de las barras
bus_results = net.res_bus

# Definir manualmente los tipos de barras basado en la configuración de la red
tipo_barra_manual = {bus1: "Slack", bus2: "PQ", bus1A: "PQ", bus2A: "PQ", bus3A: "PQ", bus1B: "PQ", bus2B: "PQ"}

# Iterar sobre cada barra y almacenar sus resultados en el DataFrame
rows_list = []
for i, bus in net.bus.iterrows():
    tipo_barra = tipo_barra_manual.get(bus.name, "PQ")  
    p = bus_results.at[i, "p_mw"] if tipo_barra == "PQ" else "N/A"
    q = bus_results.at[i, "q_mvar"] if tipo_barra == "PQ" else "N/A"
    v = bus_results.at[i, "vm_pu"]
    angulo = bus_results.at[i, "va_degree"]
    rows_list.append({"Tipo de Barra": tipo_barra, "P (MW)": p, "Q (MVAr)": q, "V (p.u.)": v, "Ángulo (°)": angulo})

barra_resultados = pd.DataFrame(rows_list)
# Mostrar el DataFrame de resultados
print("Resultados de las Barras:")
print(barra_resultados)


### Newton Raph

In [45]:
import numpy as np

#Función para calcular las derivadas parciales H, N, M, L según las definiciones dadas
def calcular_derivadas(V, theta, Y_real, Y_imag):
    n = 7 #len(V)
    H = np.zeros((n, n))
    N = np.zeros((n, n))
    M = np.zeros((n, n))
    L = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i != j:
                H[i, j] = -V[i] * V[j] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))
                N[i, j] = -V[i] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
                M[i, j] = V[i] * V[j] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
                L[i, j] = -V[i] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))
            else:
                H[i, i] = Y_imag[i, i] * V[i]**2
                N[i, i] = -Y_real[i, i] * V[i]
                M[i, i] = -Y_imag[i, i] * V[i]
                L[i, i] = -Y_real[i, i] * V[i]**2

    return H, N, M, L

#Función para calcular el vector f(x) (delta P y delta Q)
def calcular_f(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado):
    n = 7 #len(V)
    delta_P = np.zeros(n)
    delta_Q = np.zeros(n)

    for i in range(n):
        sum_P = 0.0
        sum_Q = 0.0
        for j in range(n):
            sum_P += V[i] * V[j] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
            sum_Q += V[i] * V[j] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))

        delta_P[i] = P_inyectado[i] - sum_P
        delta_Q[i] = Q_inyectado[i] - sum_Q

    f_x = np.concatenate((delta_P, delta_Q))
    return f_x
    #Función para calcular el Jacobiano J según las definiciones dadas
def calcular_jacobiano(V, theta, Y_real, Y_imag):
    n = 7 #len(V)
    H, N, M, L = calcular_derivadas(V, theta, Y_real, Y_imag)

    J = np.zeros((2 * n, 2 * n))

    # Matrices H, N, M, L en el Jacobiano J
    J[:n, :n] = H
    J[:n, n:] = N
    J[n:, :n] = M
    J[n:, n:] = L

    return J

#Función para ejecutar el método de Newton-Raphson
def newton_raphson(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado, max_iter=10, tol=1e-6):
    n = 7 #len(V)
    iteracion = 0
    delta = np.inf

    while delta > tol and iteracion < max_iter:
        # Calcular el Jacobiano J
        J = calcular_jacobiano(V, theta, Y_real, Y_imag)

        # Calcular el vector f(x) (delta P y delta Q)
        f_x = calcular_f(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado)

        # Calcular delta_x usando la inversa de J
        delta_x = -np.linalg.solve(J, f_x)

        # Actualizar V y theta
        V += delta_x[:n]
        theta += delta_x[n:]

        # Calcular la norma del vector f(x) para criterio de convergencia
        delta = np.linalg.norm(f_x)

        # Incrementar contador de iteraciones
        iteracion += 1

    if delta <= tol:
        print(f"Convergencia alcanzada en {iteracion} iteraciones.")
    else:
        print("El método de Newton-Raphson no convergió después de", max_iter, "iteraciones.")

    return V, theta